In [10]:
from disabled_passenger_api import basic_loader
import datetime as dt

In [11]:
date = "20210401"
date_list = []
date = dt.datetime.strptime(date,"%Y%m%d")

for i in range(365):
    date_str = dt.datetime.strftime(date, "%Y%m%d")
    date_list.extend([date_str])
    date = date + dt.timedelta(days=1)

In [ ]:
for i in date_list:
    data = basic_loader(i)

In [47]:
from my_azure_storage import *

import pandas as pd
import numpy as np
import itertools
import re 
import warnings 

warnings.filterwarnings("ignore")

extradata_ct = connect_container("extradata")

##행정구역 데이터
hjd_20180401 = load_json_trans_data("hjd_20180401", extradata_ct, type = "gpd")

#운영지역인 "서울", "인천", "경기"만 추출
def hjd_filter(data):
    #11 서울, 23 인천, 31 경기 
    return (data[:2] == "11") | (data[:2] == "23") | (data[:2] == "31")

hjd_20180401 = hjd_20180401.loc[list(map(lambda data: hjd_filter(data) ,hjd_20180401.adm_cd))]


# "방화 제3동' 처럼 숫자 앞 "제"가 들어가 있는 것을 "방화3동"으로 변경, 
# 정규표현식으로 [제+숫자] 패턴이 있으면 "제"를 제거 후 위치 변수를 생성해준다 
def generate_places_name(main_category, middle_category):
    #변경된 행정구역명 변수 생성 
    change_dict = {"명륜1가동": "혜화동", "명륜2가동":"혜화동", "명륜3가동":"혜화동", "명륜4가동":"혜화동", 
                "명륜5가동":"혜화동","답십리3동":"답십리1동","답십리4동":"답십리2동","장안3동":"장안2동", 
                "장안3동":"장안1동","제기1동":"제기동", "제기2동":"제기동","장안4동":"장안2동","신설동":"용신동",
                "전농3동":"전농2동","공릉1.3동":"공릉1동", "이문3동":"이문2동","고촌면":"고촌읍", "소사본1동":"소사본동",
                "양촌면":"양촌읍", "용두동":"용신동", "신당3동": "신당동", "신당4동":"신당동" ,"신당6동":"신당동",
                "신당2동":"신당동", "신당1동":"신당동", "김포1동":"김포본동", "김포2동":"장기본동", "지금동":"다산2동", "도농동":"다산2동",
                "가능2동":"흥선동", "가능3동":"흥선동", "소사본1동":"소사본동", "소사본2동":"소사본동"}
    
    p = re.compile("제+[0-9]")
    mask =  p.findall(middle_category)
    mask = [mask[0][1:]] if len(list(itertools.chain(*mask))) > 2 else mask
    if len(mask) > 0:
        middle_category = middle_category.split(mask[0])[0] + mask[0][1] + middle_category.split(mask[0])[1]
    try: 
        middle_category = change_dict[middle_category]
    except:
        pass
    category = middle_category + " " + main_category
    return category

#데이터에 맞는 법정동 geometry 리스트 반환
def get_location_bjd_geometry_and_admname(location_list, hjd_df):
    HJD_Dong_2018 = list(map(lambda data: data.split(" ")[-1], hjd_df.adm_nm))
    HJD_Sigon_2018 = list(map(lambda data: data.split(" ")[1], hjd_df.adm_nm))

    adm_name = []
    for i in location_list:
        place = i.split(" ")
        step1_mask = np.where(np.array(HJD_Dong_2018) == place[0])[0].tolist()
        if len(step1_mask) == 1:
            adm_name.append(hjd_df.iloc[step1_mask[0]].adm_nm)
        else:
            step2_mask = np.where(np.array(HJD_Sigon_2018) == place[1])[0].tolist()
            step2_mask = set(step1_mask) & set(step2_mask)
            adm_name.append(hjd_df.iloc[list(step2_mask)[0]].adm_nm)
            
    return  adm_name


def add_ps_location(disabled_data, hjd_20180401 = hjd_20180401):
    #출발지 리스트로 정의
    start_location = list(map(lambda data: generate_places_name(disabled_data.iloc[data]["startpos1"], disabled_data.iloc[data]["startpos2"]), range(len(disabled_data))))
    disabled_data["startpos"] = [i.replace(".","·") if "." in i else i for i in start_location]
    
    start_result = get_location_bjd_geometry_and_admname(disabled_data["startpos"], hjd_20180401)
    disabled_data["adm_nm_start"] = start_result
    
    return disabled_data

In [48]:
da = add_ps_location(data)

In [52]:
da[["startpos1", "startpos2", "receipttime_date", "receipttime_time"]]

,startpos1,startpos2,receipttime_date,receipttime_time
0,구로구,오류제2동,2022-03-31,01:04:05
1,송파구,오금동,2022-03-31,01:17:08
2,강남구,대치4동,2022-03-31,02:04:00
3,관악구,행운동,2022-03-31,02:24:00
4,강남구,압구정동,2022-03-31,02:37:00
...,...,...,...,...
4121,강북구,번제1동,2022-03-31,23:22:44
4122,용산구,한강로동,2022-03-31,23:30:55
4123,송파구,오금동,2022-03-31,23:36:00
4124,노원구,상계3.4동,2022-03-31,23:45:00


In [49]:
test = da[["cartype", "receipttime_date", "receipttime_time", "adm_nm_start"]]

In [13]:
import pandas as pd
import numpy as np
import warnings 
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")




### 4. 시간 단위 변환 함수 (0~1440분) 
# - 배차는 매 분마다 수행이 되도록
# - 매초단위로 수행할 경우 최적화를 돌리는게 큰 의미가 없음
def time_transform(ps_loc_data):
    #사용하는 컬럼만 copy
    passenger_df = ps_loc_data[['no', 
                                'cartype',
                                'passenger_type',
                                'settime_date',
                                'settime_time',
                                'receipttime_date',
                                'receipttime_time',
                                'start_point',
                                'end_point',
                                "adm_nm_start",
                                "adm_nm_end"]].copy()
    
    #택시가 콜 수락한 시간
    settime_date = pd.to_datetime(passenger_df["settime_date"], format = '%Y-%m-%d')
    settime_date = list(map(lambda data: data.day, settime_date))
    settime_time = pd.to_datetime(passenger_df['settime_time'], format='%H:%M:%S')
    
    #고객 콜 접수 시간
    receipttime_date = pd.to_datetime(passenger_df['receipttime_date'], format='%Y-%m-%d')
    receipttime_date = list(map(lambda data: data.day, receipttime_date))
    receipttime_time = pd.to_datetime(passenger_df['receipttime_time'], format='%H:%M:%S')
    
    #시분초 -> 0~1440분으로 변환
    set_time = settime_time.dt.minute + settime_time.dt.hour*60
    set_time = set_time.tolist()
    
    receipt_time = receipttime_time.dt.minute + receipttime_time.dt.hour*60
    receipt_time = receipt_time.tolist()
    
    passenger_df['time'] = [t+1440 if d == 17 else t for t,d in zip(receipt_time, receipttime_date)]
    passenger_df['set_time'] = [t+1440 if d == 17 else t for t,d in zip(set_time, settime_date)]
    
    passenger_df.drop(['settime_date','settime_time','receipttime_date','receipttime_time'], axis=1, inplace=True)
    passenger_df.reset_index(drop=True, inplace=True)
    return passenger_df


### 6.택시 운행 정보 데이터 추출
def generate_taxi_inf(ps_loc_data):
    taxi_start_end_dict = dict()
    for i in ps_loc_data.groupby("no"):
        taxi_start_end_dict[i[0]] = {"start":min(i[1]["set_time"])}
    
    taxi_inf = pd.DataFrame(taxi_start_end_dict.values())
    taxi_inf["no"] = taxi_start_end_dict.keys()
    
    np.random.seed(1205)
    taxi_inf["cartype"] = np.random.choice(2 ,size = len(taxi_inf), p=[0.2, 0.8])
    taxi_inf = taxi_inf[["no", "cartype", "start"]]
    return taxi_inf

### 7. 택시 근무시간 고려 출퇴근시간 임의 지정
def operation_inf(taxi_information):
    #A조 17시 이전 근무자 9시간 근무
    #B조 17시 이후 근무자 12시간 근무 
    bins = [i*60 for i in range(6,31)]
    labels = [i for i in range(6,30)]

    taxi_information["start_time"] = pd.cut(taxi_information["start"], bins, labels = labels)
    
    A = taxi_information.loc[np.array(taxi_information["start_time"].tolist()) < 17]
    B = taxi_information.loc[np.array(taxi_information["start_time"].tolist()) >= 17]
    A["start_time"] = np.array(A.start_time.tolist()) * 60
    B["start_time"] = np.array(B.start_time.tolist()) * 60
    
    A["end_time"] = np.array(A.start_time.tolist()) + (9*60)
    B["end_time"] = np.array(B.start_time.tolist()) + (12*60)
    taxi_information = pd.concat([A,B])
    
    taxi_information = taxi_information[["no", "cartype", "start_time", "end_time"]]
    return taxi_information

def estimate_taxi_schedule_plot(taxi_inf):
    bins = [i*60 for i in range(6,31)]
    labels = [f"{i}" for i in range(6,30)]
    labels_name = [f"{i}" for i in range(6,25)] + [f"{i}" for i in range(1,6)]

    start = pd.DataFrame(pd.cut(taxi_inf["start_time"], bins, labels = labels).value_counts(sort=False))
    end = pd.DataFrame(pd.cut(taxi_inf["end_time"], bins, labels = labels).value_counts(sort=False))

    f, axes = plt.subplots(1, 2)
    f.set_size_inches((22, 12))
    plt.rc('font', size=20)        # 기본 폰트 크기
    plt.rc('axes', labelsize=20)   # x,y축 label 폰트 크기

    axes[0].barh(start.index ,start["start_time"])
    axes[0].set_yticklabels(labels_name, fontsize=20)
    axes[0].set_xlim(0, 200)
    axes[0].set_xlabel("차량 대수")
    axes[0].set_ylabel("시간")
    axes[0].set_title("추정 운행 시작 시간 분포")

    axes[1].barh(end.index ,end["end_time"], color = "red")
    axes[1].set_yticklabels(labels_name, fontsize=20)
    axes[1].set_xlim(0, 200)
    axes[1].set_xlabel("차량 대수")
    axes[1].set_ylabel("시간")
    axes[1].set_title("추정 운행 종료 시간 분포")
    plt.savefig("./result_data/추정운행정보.png")


### 8. 승객 데이터, 택시 데이터 컬럼명 재정의 및 필요 컬럼 추가: 추후 혼동 방지
def redefine_col_name(ps_loc_data, taxi_loc_data): 
    # 컬럼명 변경
    ps_loc_data.columns = ['no', 'cartype', 'ps_type', 'ps_loc_0', 'ps_loc_1', 'adm_nm_start', 'adm_nm_end','call_time', 'set_time', 'ps_route', 'ps_timestamp', 'ps_distance']
    taxi_loc_data.columns = ['no', 'cartype', 'work_start', 'work_end', 'board_status', 'tx_loc']
    # 전처리 후 필요한 컬럼만 추출
    ps_loc_data = ps_loc_data[['ps_type', 'ps_loc_0', 'ps_loc_1', 'call_time', "adm_nm_start", "adm_nm_end", 'ps_route', 'ps_timestamp', 'ps_distance']]
    taxi_loc_data = taxi_loc_data[['no', 'cartype', 'work_start', 'work_end', 'board_status', 'tx_loc']]
    #콜 잡고 차량 올때까지 대기시간
    ps_loc_data["wait_time"] = 0
    #콜 잡히는데 걸리는 시간
    ps_loc_data["dispatch_time"] = 0
    #콜 실패 유무 0:성공, 1:실패 -> 지금은 실패가 없어서 알고리즘 업데이트 아직 x
    ps_loc_data["request_fail"] = 0
    #택시 승객 탑승 유무 (0:탑승, 1:미탑승)
    taxi_loc_data["board_status"] = 1
    return ps_loc_data, taxi_loc_data


### *전처리 메인 함수
def dispatch_data_preprocessing(ps_loc_data, taxi_loc_data): 
    # 승객 전처리
    ps_loc_data = cartype_transform(ps_loc_data)
    ps_loc_data = change_taxi_id(ps_loc_data)
    ps_loc_data = add_passenger_type(ps_loc_data)
    ps_loc_data = time_transform(ps_loc_data)
    ps_loc_data = get_route_time_dataframe(ps_loc_data)

    # 택시 전처리
    taxi_inf = generate_taxi_inf(ps_loc_data)
    taxi_schedule = operation_inf(taxi_inf)
    estimate_taxi_schedule_plot(taxi_schedule) #데이터 기반 택시 운행 추정
    taxi_loc_data["no"] = list(taxi_schedule.no)
    taxi_loc_data = taxi_loc_data.drop(["Taxi_ID"], axis=1)
    taxi_loc_data = pd.merge(taxi_schedule,taxi_loc_data)

    # 승객, 택시 컬럼 추가 및 재정의
    ps_loc_data, taxi_loc_data = redefine_col_name(ps_loc_data, taxi_loc_data)
    return ps_loc_data, taxi_loc_data


,no,cartype,startpos1,startpos2,endpos1,endpos2,receipttime_date,receipttime_time,settime_date,settime_time,ridetime_date,ridetime_time
0,8267,중형승합,구로구,오류제2동,도봉구,창제4동,2022-03-31,01:04:05,2022-03-31,01:34:58,2022-03-31,01:59:34
1,8049,중형 승합,송파구,오금동,남양주시,화도읍,2022-03-31,01:17:08,2022-03-31,01:25:43,2022-03-31,01:59:49
2,8270,중형승합,강남구,대치4동,도봉구,도봉제2동,2022-03-31,02:04:00,2022-03-31,02:11:40,2022-03-31,02:29:25
3,6374,중형 승합,관악구,행운동,서대문구,충현동,2022-03-31,02:24:00,2022-03-31,02:27:26,2022-03-31,02:53:42
4,7633,중형승합,강남구,압구정동,성동구,왕십리제2동,2022-03-31,02:37:00,2022-03-31,02:45:06,2022-03-31,02:58:57
...,...,...,...,...,...,...,...,...,...,...,...,...
4121,3674,중형 승합,강북구,번제1동,강북구,송천동,2022-03-31,23:22:44,2022-03-31,23:58:48,2022-04-01,00:24:42
4122,821,중형승합,용산구,한강로동,동작구,신대방제2동,2022-03-31,23:30:55,2022-04-01,00:10:34,2022-04-01,00:24:34
4123,7905,중형승합,송파구,오금동,강남구,청담동,2022-03-31,23:36:00,2022-04-01,00:52:48,2022-04-01,01:12:06
4124,7598,중형승합,노원구,상계3.4동,노원구,중계4동,2022-03-31,23:45:00,2022-04-01,00:05:02,2022-04-01,00:37:37
